In [ ]:
!pip install pyserial

In [57]:
import serial
import serial.tools.list_ports
import time
import binascii

List all the ports. Select the one of the serial converter you are using

In [147]:
# Get a list of all available serial ports
ports = serial.tools.list_ports.comports()

# Print information about each port
if not ports:
    print("No serial ports found.")
else:
    print("Available serial ports:")
    for port in ports:
        print(f"- {port.device}")
        print(f"  Description: {port.description}")
        print(f"  Hardware ID: {port.hwid}")
        print()


Available serial ports:
- COM16
  Description: USB-SERIAL CH340 (COM16)
  Hardware ID: USB VID:PID=1A86:7523 SER= LOCATION=1-3

- COM4
  Description: Standard Serial over Bluetooth link (COM4)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0002\7&17EE80CD&0&B46C4757ADF4_C00000000

- COM5
  Description: Standard Serial over Bluetooth link (COM5)
  Hardware ID: BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&17EE80CD&0&000000000000_00000000



In [148]:
# Initialize serial connection
ser = serial.Serial('COM16', 57600, timeout=1)  # Adjust baud rate if needed

# Read everything

In [ ]:
import serial
import binascii

def process_data(data):
    # Add your data processing logic here if needed
    pass

def read_serial_data():
    buffer = bytearray()
    previous_data = None
    print("Listening for serial data...")

    try:
        while True:
            byte = ser.read(1)
            if not byte:
                continue

            buffer.extend(byte)

            # Process the buffer as soon as we receive a byte
            hex_string = binascii.hexlify(buffer).decode('ascii')
            
            # Format and color the output
            formatted_output = format_and_color_data(hex_string, previous_data)
            print(f"Data: {formatted_output}")
            
            # Process the data here if needed
            process_data(buffer)
            
            # Update previous_data
            previous_data = hex_string
            
            # Clear the buffer
            buffer.clear()

            # Optional: Add a maximum buffer size check to prevent memory issues
            if len(buffer) > 1024:  # Adjust this value as needed
                print("Buffer overflow, clearing buffer")
                buffer.clear()

    except KeyboardInterrupt:
        print("\nProgram terminated by user")
    finally:
        print("Stopping data reading")

def format_and_color_data(hex_string, previous_data):
    hex_pairs = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]
    
    if previous_data is None:
        # First data, print all in blue
        return ' '.join([f"\033[44m{pair}\033[0m" for pair in hex_pairs])
    
    prev_pairs = [previous_data[i:i+2] for i in range(0, len(previous_data), 2)]
    
    colored_output = []
    for i, pair in enumerate(hex_pairs):
        if i < len(prev_pairs) and pair == prev_pairs[i]:
            colored_output.append(f"\033[44m{pair}\033[0m")  # Blue for same
        else:
            colored_output.append(f"\033[41m{pair}\033[0m")  # Red for different
    
    return ' '.join(colored_output)

# Initialize serial connection (you'll need to set the correct port and baud rate)
#ser = serial.Serial('COM19', 57600, timeout=1)  # Adjust port and baud rate if needed

# Run this function to start reading serial data
# To stop, interrupt the kernel (usually a stop button in the Jupyter interface)
read_serial_data()

# Read only commands (beginnign with FA)

In [151]:
def fletcher8(data):
    sum1 = 0
    sum2 = 0
    for byte in data:
        sum1 = (sum1 + byte) % 255
        sum2 = (sum2 + sum1) % 255
    return (sum2 << 8) | sum1

def process_command(command):
    # Add your command processing logic here
    # This function is called for each complete command received
    
    # message = command[:-2]
    # sum = 0
    # for byte in message:
    #     sum = (sum+byte)%0xff

    # crc1 = (sum - 0x89)%0xff
    

    # if sum == command[-1] and crc1 == command[-2]:
    #     print("CRC OK")
    # else:
    #     print(f"CRC NOT OK 0x{crc1:02x} 0x{sum:02x}")
    
    #---
    crc_real = command[-2]<<8 + command[-1]
    crc = fletcher8(command[:-2])
    if crc == crc_real:
        print("CRC OK")
    else:
        print(f"CRC NOT OK 0x{crc:02x}")
    #---
    
    # value = command[6]
        
    # b7 = (value + 0x27)%0xFF
    # b8 = (value + 0x11)%0xFF
    
    
    # if b7 == command[7] and b8 == command[8]:
    #     print("CRC OK")
    # else:
    #     print(f"CRC NOT OK 0x{b7:02x} 0x{b8:02x}")
    # pass

    ##LDL (command is 40)
    # b7 = (value - 0x48)%0xff
    # b8 = (value + 0x41)%0xff

    ## LDR (command is 20)
    # b7 = (value + 0x57)%0xFF
    # b8 = (value + 0x21)%0xFF
    
    ## ELR (command is 10)
    # b7 = (value + 0x27)%0xFF
    # b8 = (value + 0x11)%0xFF

    
        

def read_commands():
    buffer = bytearray()
    previous_command = None
    print("Listening for commands...")

    try:
        while True:
            byte = ser.read(1)
            if not byte:
                continue

            buffer.extend(byte)

            # Check if we have at least 2 bytes and they form a new header
            if len(buffer) >= 2 and buffer[-2:] == b'\xfa\x00':
                # If we already had a command in the buffer, print it
                if len(buffer) > 2:
                    command = buffer[:-2]  # Exclude the new header
                    hex_string = binascii.hexlify(command).decode('ascii')
                    
                    # Format and color the output
                    formatted_output = format_and_color_command(hex_string, previous_command)
                    print(f"Command: {formatted_output}")
                    
                    # Process the command here if needed
                    process_command(command)
                    
                    # Update previous_command
                    previous_command = hex_string
                    
                    # Clear the buffer, keeping only the new header
                    buffer = buffer[-2:]
            
            # Optional: Add a maximum buffer size check to prevent memory issues
            if len(buffer) > 1024:  # Adjust this value as needed
                print("Buffer overflow, clearing buffer")
                buffer.clear()

    except KeyboardInterrupt:
        print("\nProgram terminated by user")
    finally:
        print("Stopping command reading")

def format_and_color_command(hex_string, previous_command):
    hex_pairs = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]
    
    if previous_command is None:
        # First command, print all in blue
        return ' '.join([f"\033[44m{pair}\033[0m" for pair in hex_pairs])
    
    prev_pairs = [previous_command[i:i+2] for i in range(0, len(previous_command), 2)]
    
    colored_output = []
    for pair, prev_pair in zip(hex_pairs, prev_pairs):
        if pair == prev_pair:
            colored_output.append(f"\033[44m{pair}\033[0m")  # Blue for same
        else:
            colored_output.append(f"\033[41m{pair}\033[0m")  # Red for different
    
    return ' '.join(colored_output)

In [152]:
# Run this cell to start reading commands
# To stop, interrupt the kernel (usually a stop button in the Jupyter interface)
read_commands()

Listening for commands...
Command: fa 00 04 01 40 01 43 fa 84
CRC NOT OK 0xfa84

Program terminated by user
Stopping command reading


### Checking checksum

In [134]:
# checksum alogirth with byte manipulation

def xor_checksum(data):
    checksum = 0
    for byte in data:
        checksum ^= byte
    return checksum

def fletcher8(data):
    sum1 = 0
    sum2 = 0
    for byte in data:
        sum1 = (sum1 + byte) % 255
        sum2 = (sum2 + sum1) % 255
    return (sum2 << 8) | sum1

def rotating_xor(data):
    checksum = 0
    for byte in data:
        checksum = ((checksum << 1) | (checksum >> 7)) & 0xFF  # Rotate left
        checksum ^= byte
    return checksum

def fnv1a_8bit(data):
    hash_value = 0x811C9DC5
    for byte in data:
        hash_value ^= byte
        hash_value = (hash_value * 0x01000193) & 0xFF
    return hash_value

def pearson_hash(data):
    T = list(range(256))  # This should be a randomized permutation in practice
    h = 0
    for byte in data:
        h = T[h ^ (byte)]
    return h

In [144]:
command = [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x63,0x6b,0x66]#,0xfa,0x24]
command = [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x64,0x6b,0x65]#,0xfc,0x24]
command = [0xfa, 0x00, 0x04, 0x01, 0x40, 0x01, 0xa0]#, 0xb7, 0x41]
command = [0xfa, 0x00, 0x04, 0x01, 0x40, 0x01, 0xc0]#, 0x78, 0x02]
print(f"0x{xor_checksum(command):02x}")
print(f"0x{fletcher8(command):02x}")
print(f"0x{rotating_xor(command):02x}")
print(f"0x{fnv1a_8bit(command):02x}")
print(f"0x{pearson_hash(command):02x}")

0x7e
0x7802
0x35
0x91
0x7e


# It's fletcher algorithm!

In [142]:
from operator import xor
command = [0xfa, 0x00, 0x04, 0x01, 0x40, 0x01, 0xa0, 0xb7, 0x41]
print(f"0x{command[0]:02x}")



value = command[6]
print(f"0x{value:02x}")


b7 = (value - 0x48)%0xff
b8 = (value + 65)%0xff


print(f"0x{b7:02x} 0x{b8:02x}")

#testing CRC sum

# commands = [[0xfa, 0x00, 0x04, 0x01, 0x40, 0x01, 0xc0, 0x78, 0x02],
#             [0xfa, 0x00, 0x04, 0x01, 0x20, 0x01, 0xff, 0x57, 0x21],
#             [0xfa, 0x00, 0x04, 0x01, 0x20, 0x01, 0x00, 0x57, 0x21],
#             [0xfa, 0x00, 0x04, 0x01, 0x10, 0x01, 0x89, 0xb0, 0x9a],
#             [0xfa, 0x00, 0x02, 0x43, 0x04, 0x77, 0x44],
#             [0xfa, 0x00, 0x02, 0x43, 0x00, 0x73, 0x40]]

commands = [[0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x62,0x64,0x6b,0x66,0x02,0x26],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x61,0x61,0x64,0x6b,0x66,0x03,0x26],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x64,0x6b,0x66,0xfd,0x25],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x62,0x64,0x6b,0x65,0x01,0x25],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x62,0x64,0x6b,0x66,0x02,0x26],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x64,0x6b,0x66,0xfd,0x25],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x64,0x6b,0x65,0xfc,0x24],
            [0xfa,0x00,0x09,0x01,0x00,0x14,0x14,0x60,0x61,0x63,0x6b,0x66,0xfa,0x24]]


for command in commands:
    print("---")
    
    # relation between b7 and b8. is it linear?
    # b7 = command[-2]
    # b8 = command [-1]
    # sum = b7 + b8
    # sub = b7 - b8
    # sub2 = b8 - b7
    # print(f"{sum:02x}, {sub:02x}, {sub2:02x}")
    
    crc1_true = command[-2]
    crc2_true = command[-1]
    crc1 = 0
    crc2 = 0
    data2check = command[:-2]
    for byte in data2check:
        crc1 = xor(crc1,byte) 
        crc2 = (crc2 + byte) % 0xff
    
    crc = fletcher8(data2check)
    crc1 = crc >> 8
    crc2 = crc & 0x00ff
    
    if crc1 == crc1_true:
        print(f"CRC1 OK 0x{crc1:02x}")
    else:
        print(f"CRC1 Wrong 0x{crc1:02x} should be -> 0x{crc1_true:02x}")
    
    if crc2 == crc2_true:
        print(f"CRC2 OK 0x{crc2:02x}")
    else:
        print(f"CRC2 Wrong 0x{crc2:02x}  should be -> 0x{crc2_true:02x}")
    
    
    # print(f"0x{crc2:02x}, 0x{crc2:02x}")
    
#Lastbyte = iteration and sum of previous bytes (except last-1) doing mod 0xff - comfirmed



0xfa
0xa0
0x58 0xe1
---
CRC1 OK 0x02
CRC2 OK 0x26
---
CRC1 OK 0x03
CRC2 OK 0x26
---
CRC1 OK 0xfd
CRC2 OK 0x25
---
CRC1 OK 0x01
CRC2 OK 0x25
---
CRC1 OK 0x02
CRC2 OK 0x26
---
CRC1 OK 0xfd
CRC2 OK 0x25
---
CRC1 OK 0xfc
CRC2 OK 0x24
---
CRC1 OK 0xfa
CRC2 OK 0x24


In [100]:
commands_hex = [
    "fa 00 04 01 40 01 00 b7 41",
"fa 00 04 01 40 01 01 b8 42",
"fa 00 04 01 40 01 04 bb 45",
"fa 00 04 01 40 01 07 be 48",
"fa 00 04 01 40 01 09 c0 4a",
"fa 00 04 01 40 01 07 be 48",
"fa 00 04 01 40 01 04 bb 45",
"fa 00 04 01 40 01 01 b8 42",
"fa 00 04 01 40 01 00 b7 41",
"fa 00 04 01 40 01 00 b7 41",
"fa 00 04 01 40 01 01 b8 42",
"fa 00 04 01 40 01 04 bb 45",
"fa 00 04 01 40 01 07 be 48",
"fa 00 04 01 40 01 09 c0 4a",
"fa 00 04 01 40 01 0c c3 4d",
"fa 00 04 01 40 01 0e c5 4f",
"fa 00 04 01 40 01 11 c8 52",
"fa 00 04 01 40 01 14 cb 55",
"fa 00 04 01 40 01 16 cd 57",
"fa 00 04 01 40 01 19 d0 5a",
"fa 00 04 01 40 01 1b d2 5c",
"fa 00 04 01 40 01 1e d5 5f",
"fa 00 04 01 40 01 21 d8 62",
"fa 00 04 01 40 01 23 da 64",
"fa 00 04 01 40 01 26 dd 67",
"fa 00 04 01 40 01 28 df 69",
"fa 00 04 01 40 01 2b e2 6c",
"fa 00 04 01 40 01 2e e5 6f",
"fa 00 04 01 40 01 30 e7 71",
"fa 00 04 01 40 01 33 ea 74",
"fa 00 04 01 40 01 35 ec 76",
"fa 00 04 01 40 01 38 ef 79",
"fa 00 04 01 40 01 3b f2 7c",
"fa 00 04 01 40 01 3d f4 7e",
"fa 00 04 01 40 01 40 f7 81",
"fa 00 04 01 40 01 43 fa 84",
"fa 00 04 01 40 01 45 fc 86",
"fa 00 04 01 40 01 48 00 89",
"fa 00 04 01 40 01 4a 02 8b",
"fa 00 04 01 40 01 4d 05 8e",
"fa 00 04 01 40 01 50 08 91",
"fa 00 04 01 40 01 52 0a 93",
"fa 00 04 01 40 01 55 0d 96",
"fa 00 04 01 40 01 57 0f 98",
"fa 00 04 01 40 01 5a 12 9b",
"fa 00 04 01 40 01 5d 15 9e",
"fa 00 04 01 40 01 5f 17 a0",
"fa 00 04 01 40 01 62 1a a3",
"fa 00 04 01 40 01 64 1c a5",
"fa 00 04 01 40 01 67 1f a8",
"fa 00 04 01 40 01 6a 22 ab",
"fa 00 04 01 40 01 6c 24 ad",
"fa 00 04 01 40 01 6f 27 b0",
"fa 00 04 01 40 01 71 29 b2",
"fa 00 04 01 40 01 74 2c b5",
"fa 00 04 01 40 01 77 2f b8",
"fa 00 04 01 40 01 79 31 ba",
"fa 00 04 01 40 01 7c 34 bd",
"fa 00 04 01 40 01 7f 37 c0",
"fa 00 04 01 40 01 81 39 c2",
"fa 00 04 01 40 01 84 3c c5",
"fa 00 04 01 40 01 86 3e c7",
"fa 00 04 01 40 01 89 41 ca",
"fa 00 04 01 40 01 8c 44 cd",
"fa 00 04 01 40 01 8e 46 cf",
"fa 00 04 01 40 01 91 49 d2",
"fa 00 04 01 40 01 93 4b d4",
"fa 00 04 01 40 01 96 4e d7",
"fa 00 04 01 40 01 99 51 da",
"fa 00 04 01 40 01 9b 53 dc",
"fa 00 04 01 40 01 9e 56 df",
"fa 00 04 01 40 01 a0 58 e1",
"fa 00 04 01 40 01 a3 5b e4",
"fa 00 04 01 40 01 a6 5e e7",
"fa 00 04 01 40 01 a8 60 e9",
"fa 00 04 01 40 01 ab 63 ec",
"fa 00 04 01 40 01 ad 65 ee",
"fa 00 04 01 40 01 b0 68 f1",
"fa 00 04 01 40 01 b3 6b f4",
"fa 00 04 01 40 01 b5 6d f6",
"fa 00 04 01 40 01 b8 70 f9",
"fa 00 04 01 40 01 bb 73 fc",
"fa 00 04 01 40 01 bd 75 fe",
"fa 00 04 01 40 01 c0 78 02",
"fa 00 04 01 40 01 c2 7a 04",
"fa 00 04 01 40 01 c5 7d 07",
"fa 00 04 01 40 01 c8 80 0a",
"fa 00 04 01 40 01 ca 82 0c",
"fa 00 04 01 40 01 cd 85 0f",
"fa 00 04 01 40 01 cf 87 11",
"fa 00 04 01 40 01 d2 8a 14",
"fa 00 04 01 40 01 d5 8d 17",
"fa 00 04 01 40 01 d7 8f 19",
"fa 00 04 01 40 01 da 92 1c",
"fa 00 04 01 40 01 dc 94 1e",
"fa 00 04 01 40 01 df 97 21",
"fa 00 04 01 40 01 e2 9a 24",
"fa 00 04 01 40 01 e4 9c 26",
"fa 00 04 01 40 01 e7 9f 29",
"fa 00 04 01 40 01 e9 a1 2b",
"fa 00 04 01 40 01 ec a4 2e",
"fa 00 04 01 40 01 ef a7 31",
"fa 00 04 01 40 01 f1 a9 33",
"fa 00 04 01 40 01 f4 ac 36",
"fa 00 04 01 40 01 f7 af 39",
"fa 00 04 01 40 01 f9 b1 3b",
"fa 00 04 01 40 01 fc b4 3e",
"fa 00 04 01 40 01 fe b6 40",
"fa 00 04 01 40 01 ff b7 41",
    
    
    
    "fa 00 04 01 40 01 00 b7 41",
    "fa 00 04 01 40 01 01 b8 42",
    "fa 00 04 01 40 01 04 bb 45",
    "fa 00 04 01 40 01 07 be 48",
    "fa 00 04 01 40 01 09 c0 4a",
    "fa 00 04 01 40 01 07 be 48",
    "fa 00 04 01 40 01 04 bb 45",
    "fa 00 04 01 40 01 01 b8 42",
    "fa 00 04 01 40 01 00 b7 41",
    "fa 00 04 01 40 01 00 b7 41",
    "fa 00 04 01 40 01 01 b8 42",
    "fa 00 04 01 40 01 04 bb 45",
    "fa 00 04 01 40 01 07 be 48",
    "fa 00 04 01 40 01 09 c0 4a",
    "fa 00 04 01 40 01 0c c3 4d",
    "fa 00 04 01 40 01 0e c5 4f",
    "fa 00 04 01 40 01 11 c8 52",
    "fa 00 04 01 40 01 14 cb 55",
    "fa 00 04 01 40 01 16 cd 57",
    "fa 00 04 01 40 01 19 d0 5a",
    "fa 00 04 01 40 01 1b d2 5c",
    "fa 00 04 01 40 01 1e d5 5f",
    "fa 00 04 01 40 01 21 d8 62",
    "fa 00 04 01 40 01 23 da 64",
    "fa 00 04 01 40 01 26 dd 67",
    "fa 00 04 01 40 01 28 df 69",
    "fa 00 04 01 40 01 2b e2 6c",
    "fa 00 04 01 40 01 2e e5 6f",
    "fa 00 04 01 40 01 30 e7 71",
    "fa 00 04 01 40 01 33 ea 74",
    "fa 00 04 01 40 01 35 ec 76",
    "fa 00 04 01 40 01 38 ef 79",
    "fa 00 04 01 40 01 3b f2 7c",
    "fa 00 04 01 40 01 3d f4 7e",
    "fa 00 04 01 40 01 40 f7 81",
    "fa 00 04 01 40 01 43 fa 84",
    "fa 00 04 01 40 01 45 fc 86",
    "fa 00 04 01 40 01 48 00 89",
    "fa 00 04 01 40 01 4a 02 8b",
    "fa 00 04 01 40 01 4d 05 8e",
    "fa 00 04 01 40 01 50 08 91",
    "fa 00 04 01 40 01 52 0a 93",
    "fa 00 04 01 40 01 55 0d 96",
    "fa 00 04 01 40 01 57 0f 98",
    "fa 00 04 01 40 01 5a 12 9b",
    "fa 00 04 01 40 01 5d 15 9e",
    "fa 00 04 01 40 01 5f 17 a0",
    "fa 00 04 01 40 01 62 1a a3",
    "fa 00 04 01 40 01 64 1c a5",
    "fa 00 04 01 40 01 67 1f a8",
    "fa 00 04 01 40 01 6a 22 ab",
    "fa 00 04 01 40 01 6c 24 ad",
    "fa 00 04 01 40 01 6f 27 b0",
    "fa 00 04 01 40 01 71 29 b2",
    "fa 00 04 01 40 01 74 2c b5",
    "fa 00 04 01 40 01 77 2f b8",
    "fa 00 04 01 40 01 79 31 ba",
    "fa 00 04 01 40 01 7c 34 bd",
    "fa 00 04 01 40 01 7f 37 c0",
    "fa 00 04 01 40 01 81 39 c2",
    "fa 00 04 01 40 01 84 3c c5",
    "fa 00 04 01 40 01 86 3e c7",
    "fa 00 04 01 40 01 89 41 ca",
    "fa 00 04 01 40 01 8c 44 cd",
    "fa 00 04 01 40 01 8e 46 cf",
    "fa 00 04 01 40 01 91 49 d2",
    "fa 00 04 01 40 01 93 4b d4",
    "fa 00 04 01 40 01 96 4e d7",
    "fa 00 04 01 40 01 99 51 da",
    "fa 00 04 01 40 01 9b 53 dc",
    "fa 00 04 01 40 01 9e 56 df",
    "fa 00 04 01 40 01 a0 58 e1",
    "fa 00 04 01 40 01 a3 5b e4",
    "fa 00 04 01 40 01 a6 5e e7",
    "fa 00 04 01 40 01 a8 60 e9",
    "fa 00 04 01 40 01 ab 63 ec",
    "fa 00 04 01 40 01 ad 65 ee",
    "fa 00 04 01 40 01 b0 68 f1",
    "fa 00 04 01 40 01 b3 6b f4",
    "fa 00 04 01 40 01 b5 6d f6",
    "fa 00 04 01 40 01 b8 70 f9",
    "fa 00 04 01 40 01 bb 73 fc",
    "fa 00 04 01 40 01 bd 75 fe",
    "fa 00 04 01 40 01 c0 78 02",
    "fa 00 04 01 40 01 c2 7a 04",
    "fa 00 04 01 40 01 c5 7d 07",
    "fa 00 04 01 40 01 c8 80 0a",
    "fa 00 04 01 40 01 ca 82 0c",
    "fa 00 04 01 40 01 cd 85 0f",
    "fa 00 04 01 40 01 cf 87 11",
    "fa 00 04 01 40 01 d2 8a 14",
    "fa 00 04 01 40 01 d5 8d 17",
    "fa 00 04 01 40 01 d7 8f 19",
    "fa 00 04 01 40 01 da 92 1c",
    "fa 00 04 01 40 01 dc 94 1e",
    "fa 00 04 01 40 01 df 97 21",
    "fa 00 04 01 40 01 e2 9a 24",
    "fa 00 04 01 40 01 e4 9c 26",
    "fa 00 04 01 40 01 e7 9f 29",
    "fa 00 04 01 40 01 e9 a1 2b",
    "fa 00 04 01 40 01 ec a4 2e",
    "fa 00 04 01 40 01 ef a7 31",
    "fa 00 04 01 40 01 f1 a9 33",
    "fa 00 04 01 40 01 f4 ac 36",
    "fa 00 04 01 40 01 f7 af 39",
    "fa 00 04 01 40 01 f9 b1 3b",
    "fa 00 04 01 40 01 fc b4 3e",
    "fa 00 04 01 40 01 fe b6 40",
    "fa 00 04 01 40 01 ff b7 41"
]

def hex_byte_to_int(h):
    return int(h,16)

mismatch_count = 0
results = []
for line in commands_hex:
    parts = line.split()
    # The motor value M is the 6th index in the 9-byte list: 0 1 2 3 4 5 [6] 7 8
    # Just to confirm structure: "fa 00 04 01 40 01 00 b7 41"
    # indexes:                0  1  2  3  4  5  6  7  8
    M_hex = parts[6]
    b7_hex = parts[7]
    b8_hex = parts[8]
    
    M = hex_byte_to_int(M_hex)
    b7_actual = hex_byte_to_int(b7_hex)
    b8_actual = hex_byte_to_int(b8_hex)
    
    # relation between b7 and b8. is it linear?
    sum = b7 + b8
    sub = b7 - b8
    sub2 = b8 - b7
    print(f"{sum:02x}, {sub:02x}, {sub2:02x}")
    
    # Proposed formula:
    # b7 = (M - 72) mod 256
    # b8 = (M + 65) mod 256
    
    b7_expected = (M - 0x48) % 0xff
    b8_expected = (M + 65) % 0xff
    
    match_b7 = (b7_actual == b7_expected)
    match_b8 = (b8_actual == b8_expected)
    match_overall = match_b7 and match_b8
    
    if not match_overall:
        mismatch_count += 1
    
    results.append((M_hex, b7_hex, b8_hex,
                    f"{b7_expected:02x}",
                    f"{b8_expected:02x}",
                    match_overall))

# Print summary
print(f"Total commands checked: {len(commands_hex)}")
print(f"Number of mismatches:   {mismatch_count}")

if mismatch_count == 0:
    print("\nConclusion: The formula b7=(M-72) mod256 and b8=(M+65) mod256 holds for ALL commands.")
else:
    print("\nConclusion: Formula does not perfectly match all commands. See mismatches below.\n")

# If mismatches, show them
if mismatch_count > 0:
    print(f"{'M':<4} {'b7':<3} {'b8':<3} {'b7exp':<5} {'b8exp':<5} {'match':<6}")
    for (mh, b7h, b8h, b7e, b8e, ok) in results:
        if not ok:
            print(f"{mh:<4} {b7h:<3} {b8h:<3} {b7e:<5} {b8e:<5} {str(ok):<6}")

139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89
139, -89, 89

# Creating the library

In [ ]:
command = bytearray([0xfa, 0x00, 0x04, 0x01, 0x40, 0x01, 0x00, 0xb7, 0x41])
ser.write(command)
#fa 00 04 01 40 01 ff b7 41



In [ ]:
import serial
import time

def move_robot_part(part, value):
    # Map parts to their respective command codes
    part_codes = {
        'LDL': 0x40,
        'LDR': 0x20,
        'ELR': 0x10,
        'EUD': 0x08,
        'NE': 0x04,
        'NR': 0x02,
        'NT': 0x01
    }

    if part not in part_codes:
        raise ValueError("Invalid part specified")

    # Map 0-100 to 0-255
    mapped_value = int(value * 2.55)

    # Construct the command
    command = bytearray([0xfa, 0x00, 0x04, 0x01, part_codes[part], 0x01, mapped_value])

    # Calculate checksum (sum of all bytes modulo 256)
    checksum = sum(command) % 256
    command.append(checksum)

    # Send the command
    ser.write(command)
    time.sleep(0.1)  # Small delay to ensure command is processed

# Initialize serial connection
# ser = serial.Serial('COM19', 57600, timeout=1)

def move_all_parts(ldl, ldr, elr, eud, ne, nr, nt):
    move_robot_part('LDL', ldl)
    move_robot_part('LDR', ldr)
    move_robot_part('ELR', elr)
    move_robot_part('EUD', eud)
    move_robot_part('NE', ne)
    move_robot_part('NR', nr)
    move_robot_part('NT', nt)

# Example usage:
move_all_parts(50, 50, 50, 50, 50, 50, 50)

In [ ]:
move_robot_part('NT', 20)